In [1]:
import pandas as pd
import numpy as np
import pyreadstat
import altair as alt

In [2]:
df, meta = pyreadstat.read_sav('ticdom_2019_individuos_base_de_microdados_v1.0.sav', encoding='latin1')

In [3]:
def convert_to_label(coluna, valor, meta):
    try:
        return meta.value_labels[meta.variable_to_label[coluna]][valor]
    except:
        return valor

In [4]:
df_labels = df.copy()

In [5]:
for c in list(df_labels):
    df_labels[c] = df_labels[c].apply(lambda x : convert_to_label(c, x, meta))

In [6]:
df_teste = df.copy()

for coluna in df_teste.columns:
    try:
        df_teste[coluna] = df_teste[coluna].map(meta.value_labels[meta.variable_to_label[coluna]])
    except:
        pass

In [7]:
import pandas as pd
import numpy as np

df1 = pd.DataFrame({'col1':['w', 10, 20],
                   'col2': ['a', 30, np.nan]},
                  index=[1,2,0])
#   col1 col2
# 1    w    a
# 2   10   30
# 0   20  NaN

df1

,col1,col2
1,w,a
2,10,30
0,20,NaN


In [8]:
dict = {0: "A", 2: "B"}

# The value at the 0-index is mapped to 'A', the value at the 2-index is mapped to 'B'
df1['col1'].update(pd.Series(dict))
print(df1)

  col1 col2
1    w    a
2    B   30
0    A  NaN


In [9]:
def weighted_frequency(x, y):
    a = pd.Series(df[[x,y]].groupby(x).sum()[y]) / df[y].sum()
    b = a.index.map(meta.variable_value_labels[x])
    c = a.values
    df_temp = pd.DataFrame({'Labels': b, 'Frequency': c})
    return df_temp

In [10]:
print(meta.column_labels[list(df_labels.columns).index('B1')])
pd.crosstab(df_labels['RACA'],
            df_labels['B1'],
            df_labels['PESO'], aggfunc='sum', normalize='index')

O respondente já usou um computador de mesa, um notebook ou um tablet?


B1,Não,Não respondeu,Não sabe,Sim
RACA,,,,
Amarela,0.429996,0.000000,0.000925,0.569079
Branca,0.373124,0.000486,0.000059,0.626331
Indígena,0.507732,0.014052,0.000273,0.477944
Não respondeu,0.661876,0.001373,0.000000,0.336751
Parda,0.427888,0.001571,0.000000,0.570541
Preta,0.451040,0.002388,0.000165,0.546408


In [11]:
def gera_tabela_frequencia(df, dimensao, indice, peso):
    return pd.crosstab(df[dimensao], df[indice], df[peso],
        aggfunc='sum',
        normalize='index')

In [12]:
df_crosstabbed = gera_tabela_frequencia(df_labels, 'SEXO', 'B1', 'PESO')

In [13]:
df_crosstabbed.reset_index(inplace=True)

In [14]:
df_crosstabbed

B1,SEXO,Não,Não respondeu,Não sabe,Sim
0,Feminino,0.450328,0.001004,0.000049,0.548619
1,Masculino,0.382605,0.001871,0.000089,0.615436


In [15]:
# MANTENHA VAR SEXO...
# TODAS AS OUTRAS COLUNAS, FICAM CONTIDAS NUMA COLUNA UNICA CHAMADA 'RESPOSTAS'
# VALUE_NAME VAI SER AS OCORRENCIAS DOS VALORE

resposta = df_crosstabbed.melt(id_vars='SEXO', var_name='RESPOSTAS', value_name='FREQUENCIA')

In [16]:
resposta

,SEXO,RESPOSTAS,FREQUENCIA
0,Feminino,Não,0.450328
1,Masculino,Não,0.382605
2,Feminino,Não respondeu,0.001004
3,Masculino,Não respondeu,0.001871
4,Feminino,Não sabe,0.000049
5,Masculino,Não sabe,0.000089
6,Feminino,Sim,0.548619
7,Masculino,Sim,0.615436


In [17]:
alt.Chart(resposta).mark_bar().encode(
    x = alt.X('RESPOSTAS', axis = alt.Axis(labelAngle=45, title='Respostas')),
    color = 'RESPOSTAS',
    column = 'SEXO',
    y = alt.Y('FREQUENCIA', axis=alt.Axis(title='Frequencia'))
).properties(width=200)

alt.Chart(...)